In [33]:
import org.apache.spark.{SparkContext, SparkConf}

println("Starting Spark Session...")

val username = "alexey_chernyaev2"
val password = "fuiBpd2S"
val hostname = "10.0.0.5"

val postgreUrl = s"jdbc:postgresql://$hostname:5432"

val conf = new SparkConf()
                    .set("spark.cassandra.connection.host", hostname)
                    .set("spark.cassandra.connection.port", "9042")
                    .set("spark.cassandra.auth.username", "alexey.chernyaev2")
                    .set("spark.cassandra.auth.password", password)
                    .set("spark.cassandra.output.consistency.level", "ANY")
                    .set("spark.cassandra.input.consistency.level", "ONE")
    
val sparkSession = SparkSession.builder()
                        .appName("lag-custom")
                        .config(conf=conf)
                        .getOrCreate()

println("Spark Session was started")





Starting Spark Session...
Spark Session was started


username = alexey_chernyaev2
password = fuiBpd2S
hostname = 10.0.0.5
postgreUrl = jdbc:postgresql://10.0.0.5:5432
conf = org.apache.spark.SparkConf@35ae572e
sparkSession = org.apache.spark.sql.SparkSession@4c4fb9dd


lastException: Throwable = null


org.apache.spark.sql.SparkSession@4c4fb9dd

# HDFS

In [34]:
val webLogsDF = spark.read.parquet("/labs/laba03/weblogs.parquet")

println(webLogsDF.cache.count)
webLogsDF.printSchema
webLogsDF.show(3, 200, true)

36138
root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- url: string (nullable = true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 uid    | 39d8aded-2111-42ab-9393-37c4f781d030                                                                                                                                                                     
 visits | [[1419590902274, http://go.mail.ru/], [1419590344372, http://thephilanews.com/ru/wawa-welcome-america-begins-with-reading-…-and-the-runway-32987.htm], [1419590203997, http://visitorsworth.com/www.w... 
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------

webLogsDF = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [35]:
import org.apache.spark.sql.functions._

val webLogsStatsDF = webLogsDF
                        .select($"uid", explode($"visits").as("visit"))
                        .withColumn("host", lower(callUDF("parse_url", $"visit.url", lit("HOST"))))
                        .withColumn("domain", regexp_replace(lower($"host"), "www.", ""))
                        .where($"uid".isNotNull)
                        .groupBy($"uid", $"domain").count

println(webLogsStatsDF.cache.count)
webLogsStatsDF.printSchema
webLogsStatsDF.show(3, 200, true)

572694
root
 |-- uid: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- count: long (nullable = false)

-RECORD 0--------------------------------------
 uid    | 3a081c72-5c55-4c7b-bf52-b56201d85b55 
 domain | videograbber.net                     
 count  | 1                                    
-RECORD 1--------------------------------------
 uid    | 3a48922d-4621-455e-ba68-f273b95807fb 
 domain | umeltsi.ru                           
 count  | 4                                    
-RECORD 2--------------------------------------
 uid    | 63046a3d-830e-44b3-b580-a16db013234d 
 domain | intvua.com                           
 count  | 2                                    
only showing top 3 rows



webLogsStatsDF = [uid: string, domain: string ... 1 more field]


[uid: string, domain: string ... 1 more field]

# postgreSql

In [7]:
%AddJar file:///data/home/alexey.chernyaev2/postgresql-42.2.12.jre6.jar

Using cached version of postgresql-42.2.12.jre6.jar


In [36]:
val domainCatsDF = spark.read
                                .format("jdbc")
                                .option("driver", "org.postgresql.Driver")
                                .option("url", s"$postgreUrl/labdata")
                                .option("dbtable", "domain_cats")
                                .option("user", username)
                                .option("password", password)
                                .load()
println(domainCatsDF.cache.count)
domainCatsDF.printSchema
domainCatsDF.show(3, 200, true)

245981
root
 |-- domain: string (nullable = true)
 |-- category: string (nullable = true)

-RECORD 0--------------------------
 domain   | national-lottery.co.uk 
 category | gambling               
-RECORD 1--------------------------
 domain   | rivalo.com             
 category | gambling               
-RECORD 2--------------------------
 domain   | xoso.me                
 category | gambling               
only showing top 3 rows



domainCatsDF = [domain: string, category: string]


[domain: string, category: string]

In [37]:
val webStatsDF = webLogsStatsDF
                        .join(domainCatsDF, Seq("domain"), "left")
                        .withColumn("web_cat", concat(lit("web_"), regexp_replace(lower($"category"), "[ -]", "_")))
                        .where($"web_cat".isNotNull)
                        .groupBy($"uid")
                        .pivot($"web_cat")
                        .sum("count")

println(webStatsDF.cache.count)
webStatsDF.printSchema
webStatsDF.show(3, 200, true)


27802
root
 |-- uid: string (nullable = true)
 |-- web_arts_and_entertainment: long (nullable = true)
 |-- web_autos_and_vehicles: long (nullable = true)
 |-- web_beauty_and_fitness: long (nullable = true)
 |-- web_books_and_literature: long (nullable = true)
 |-- web_business_and_industry: long (nullable = true)
 |-- web_career_and_education: long (nullable = true)
 |-- web_computer_and_electronics: long (nullable = true)
 |-- web_finance: long (nullable = true)
 |-- web_food_and_drink: long (nullable = true)
 |-- web_gambling: long (nullable = true)
 |-- web_games: long (nullable = true)
 |-- web_health: long (nullable = true)
 |-- web_home_and_garden: long (nullable = true)
 |-- web_internet_and_telecom: long (nullable = true)
 |-- web_law_and_government: long (nullable = true)
 |-- web_news_and_media: long (nullable = true)
 |-- web_pets_and_animals: long (nullable = true)
 |-- web_recreation_and_hobbies: long (nullable = true)
 |-- web_reference: long (nullable = true)
 |-- web_sc

webStatsDF = [uid: string, web_arts_and_entertainment: bigint ... 22 more fields]


[uid: string, web_arts_and_entertainment: bigint ... 22 more fields]

# ES

In [11]:
%AddJar file:///data/home/alexey.chernyaev2/elasticsearch-spark-20_2.11-7.7.0.jar

Using cached version of elasticsearch-spark-20_2.11-7.7.0.jar


In [40]:

val esOptions =
  Map(
    "es.nodes" -> "10.0.0.5:9200",
    "es.batch.write.refresh" -> "false",
    "es.nodes.wan.only" -> "true",
    "es.net.http.auth.user" -> "alexey.chernyaev2",
    "es.net.http.auth.pass" -> password
  )
var visitsDF = spark.read.format("org.elasticsearch.spark.sql").options(esOptions).load("visits*")

println(visitsDF.cache.count)
visitsDF.printSchema
visitsDF.show(3, 200, true)

182540
root
 |-- category: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_price: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- uid: string (nullable = true)

-RECORD 0------------------------------------------
 category   | Entertainment-equipment              
 event_type | buy                                  
 item_id    | Entertainment-equipment-2            
 item_price | 2529                                 
 timestamp  | 2020-01-01 11:00:00                  
 uid        | 40b29579-e845-45c0-a34d-03630d296a81 
-RECORD 1------------------------------------------
 category   | Everyday-jewelry                     
 event_type | buy                                  
 item_id    | Everyday-jewelry-0                   
 item_price | 4320                                 
 timestamp  | 2020-01-01 11:01:00                  
 uid        | null                                 
-RECORD 2----------

esOptions = Map(es.nodes.wan.only -> true, es.net.http.auth.user -> alexey.chernyaev2, es.net.http.auth.pass -> fuiBpd2S, es.batch.write.refresh -> false, es.nodes -> 10.0.0.5:9200)
visitsDF = [category: string, event_type: string ... 4 more fields]


[category: string, event_type: string ... 4 more fields]

In [41]:
val visitStatsDF = visitsDF
                    .filter($"uid".isNotNull)
                    .withColumn("shop_cat", concat(lit("shop_"), regexp_replace(lower($"category"), "[ -]", "_")))
                    .groupBy($"uid", $"shop_cat")
                    .count()
println(visitStatsDF.cache.count)
visitStatsDF.printSchema
visitStatsDF.show(3, 200, true)


86507
root
 |-- uid: string (nullable = true)
 |-- shop_cat: string (nullable = true)
 |-- count: long (nullable = false)

-RECORD 0----------------------------------------
 uid      | 75fcbe0c-ef8a-452f-8864-ea3a4f376ae2 
 shop_cat | shop_computers                       
 count    | 1                                    
-RECORD 1----------------------------------------
 uid      | 9e241ecb-af15-49a7-8fef-516aaae0456d 
 shop_cat | shop_cameras                         
 count    | 1                                    
-RECORD 2----------------------------------------
 uid      | 372c086a-c53d-45fd-b947-abc96612030a 
 shop_cat | shop_mobile_phones                   
 count    | 1                                    
only showing top 3 rows



visitStatsDF = [uid: string, shop_cat: string ... 1 more field]


[uid: string, shop_cat: string ... 1 more field]

# cassandra

In [42]:
%AddJar file:///data/home/alexey.chernyaev2/spark-cassandra-connector_2.11-2.4.3.jar

Using cached version of spark-cassandra-connector_2.11-2.4.3.jar


In [43]:

    val clientsOpts = Map("table" -> "clients","keyspace" -> "labdata")
    val clients = spark.read
      .format("org.apache.spark.sql.cassandra")
      .options(clientsOpts)
      .load()

Name: java.lang.NoClassDefFoundError
Message: com/datastax/spark/connector/TableRef
StackTrace:   at org.apache.spark.sql.cassandra.DefaultSource$.TableRefAndOptions(DefaultSource.scala:142)
  at org.apache.spark.sql.cassandra.DefaultSource.createRelation(DefaultSource.scala:56)
  at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:332)
  at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:242)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:230)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)

In [44]:
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql.CassandraConnector

val connector = CassandraConnector(sc.getConf)

val clientsDF = spark.read
                            .format("org.apache.spark.sql.cassandra")
                            .options(Map("table" -> "clients", "keyspace" -> "labdata"))
                            .load()
println(clientsDF.cache.count)
clientsDF.printSchema
clientsDF.show(5, 200, true)

Name: Unknown Error
Message: lastException: Throwable = null
<console>:379: error: Symbol 'term com.datastax.oss' is missing from the classpath.
This symbol is required by 'method com.datastax.spark.connector.cql.CassandraConnector.openSession'.
Make sure that term oss is in your classpath and check for conflicting dependencies with `-Ylog-classpath`.
A full rebuild may help if 'CassandraConnector.class' was compiled against an incompatible version of com.datastax.
       val clientsDF = spark.read
                       ^

StackTrace: 

In [45]:

val tableOpts = Map("table" -> "clients", "keyspace" -> "labdata")
val clients = spark
  .read
  .format("org.apache.spark.sql.cassandra")
  .options(tableOpts)
  .load()
clients.show()

Name: java.lang.NoClassDefFoundError
Message: com/datastax/spark/connector/TableRef
StackTrace:   at org.apache.spark.sql.cassandra.DefaultSource$.TableRefAndOptions(DefaultSource.scala:142)
  at org.apache.spark.sql.cassandra.DefaultSource.createRelation(DefaultSource.scala:56)
  at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:332)
  at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:242)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:230)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)

In [46]:
val spark = SparkSession
  .builder()
  .appName("Lab03")
  .config("spark.cassandra.connection.host", "10.0.0.5")
  .config("spark.cassandra.connection.port", "9042")
  .config("spark.cassandra.output.consistency.level", "ANY")
  .config("spark.cassandra.input.consistency.level", "ONE")
  .getOrCreate()
import spark.implicits._
val tableOpts = Map("table" -> "clients", "keyspace" -> "labdata")
val clients = spark
  .read
  .format("org.apache.spark.sql.cassandra")
  .options(tableOpts)
  .load()
clients.show()

lastException = null


Name: java.lang.NoClassDefFoundError
Message: com/datastax/spark/connector/TableRef
StackTrace:   at org.apache.spark.sql.cassandra.DefaultSource$.TableRefAndOptions(DefaultSource.scala:142)
  at org.apache.spark.sql.cassandra.DefaultSource.createRelation(DefaultSource.scala:56)
  at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:332)
  at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:242)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:230)
  at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)

In [ ]:
import org.apache.spark.sql.functions.when

val shopStatsDF = clientsDF
                        .withColumn("age_cat", when($"age" >= 55, ">=55")
                                              .when($"age" >= 45, "45-54")
                                              .when($"age" >= 35, "35-44")
                                              .when($"age" >= 25, "25-34")
                                              .when($"age" >= 18, "18-24"))
                        .join(visitStatsDF, Seq("uid"), "left")
                        .groupBy($"uid", $"gender", $"age_cat")
                        .pivot($"shop_cat")
                        .sum("count")
                        .drop($"null")
println(shopStatsDF.cache.count)
shopStatsDF.printSchema
shopStatsDF.show(3, 200, true)


# create DataMart

In [ ]:
val datamartDF = shopStatsDF
                        .join(webStatsDF, Seq("uid"), "left")
                        .orderBy($"uid")
println(datamartDF.cache.count)
datamartDF.printSchema
datamartDF.show(3, 200, true)


In [ ]:
val options = Map(
                 "url" -> s"$postgreUrl/$username",
                 "user" -> username,
                 "password" -> password,
                 "mode" -> "overwrite",
                 "driver" -> "org.postgresql.Driver",
                 "dbtable" -> "clients")

datamartDF.write.format("jdbc").options(options).mode("overwrite").save()